#Student Name:Ganapathy
#Student ID:18202799

In [4]:
import random

class Matrix_Create:
    def __init__(self, inn):
        self.inn = inn
    def create_all_zero_matrix(self):
        
        row = size[0]
        column = size[1]
        matrix = [0] * row
        for index in range (row):
            matrix[index] = [0] * column
        return matrix

    # create matrix of given size. Input is tuple with two elements
    # first element represent row , second element represent column
    # This method creates matrix and fill it with random numbers
    ##Combined vector & Matrix creation into single method
    def create_matrix(self, size):
        #print("Printing Size:", size)
        row = size[0]
        column = size[1]
        matrix = [0] * row
        for index in range (row):
            matrix[index] = [0] * column
            
        # populate matrix with random values
        for row_index in range(row):
            for column_index in range(column):
                matrix[row_index][column_index] = random.randint(1,5)
        return matrix

    
class Matrix:
    def __init__(self, inputA, inputB):
        self.inputA = inputA
        self.inputB = inputB
    
    def check_if_incompatible_matrixes(self, first_matrix,second_matrix):         
        if (len(first_matrix) == len(second_matrix)):
            if(len(first_matrix[0]) == len(second_matrix[0])):
                return False
            else:
                return True
        else:
            return True

    def matrix_addORsub(self, operation):
    # check if the input matrixes are compatible with same size
        if (self.check_if_incompatible_matrixes(self.inputA,self.inputB)):
            return ("Matrix not compatible!!!")
        # create result matrix 
        result_matrix = Matrix_Create.create_all_zero_matrix(self) 
        operations = ['+','-']
        if operation in operations:
            # get the rows of first matrix 
            for row in range (len(self.inputA)):
                column = 0
                # get the column of second matrix 
                for i, j in zip(self.inputA[row],self.inputB[row]):
                    if operation == '+':
                        result_matrix[row][column] = i + j
                        column += 1
                    elif operation == '-':
                        result_matrix[row][column] = i - j
                        column += 1
            return result_matrix
    
    def matrix_multiplication(self):
    # check if the input matrixes are compatible with same size
        if (self.check_if_incompatible_matrixes(self.inputA,self.inputB)):
            return ("Matrix not compatible!!!")
        # create result matrix 
        result_matrix = Matrix_Create.create_all_zero_matrix(self)
        # get the rows of first matrix
        result_matrix = [[sum(a*b for a,b in zip(self.inputA_row,self.inputB_col)) for self.inputB_col in zip(*self.inputB)] for self.inputA_row in self.inputA]
        return result_matrix
    

    def determinant(self):
        """Method to calculate determinant of 2x2 Matrix"""
        #print(self.inputA[0][0], self.inputA[0][1])
        deter_result = (self.inputA[0][0]*self.inputA[1][1]) - (self.inputA[1][0]*self.inputA[0][1])
        return deter_result

    def inverse(self):
        """Method to calculate the inverse of a 2x2 Matrix"""
        determinant = self.determinant()
        # Raise an exception if the determinant is 0 to avoid divide by zero
        inv_result = []
        if determinant == 0:
            raise Exception("Determinant is Zero")
        det_inv = 1 / determinant
        temp = self.inputA[0][0]
        self.inputA[0][0] = self.inputA[1][1]
        self.inputA[1][1] = temp

        self.inputA[1][0] *= -1
        self.inputA[0][1] *= -1

        mat_00 = self.inputA[0][0] * det_inv
        mat_01 = self.inputA[0][1] * det_inv
        mat_10 = self.inputA[1][0] * det_inv
        mat_11 = self.inputA[1][1] * det_inv
        inv_result = (((mat_11, -mat_01), (-mat_10, mat_00)))
        return inv_result
    
    def cross_product(self, cross_product_inputA, cross_product_inputB ):
        """Method to calculate the cross product of two vectors of suitable size using the Right Hand Rule
        (a_x, a_y, a_z) x (b_x, b_y, b_z) = (a_y.b_z - a_z.b_y, a_z.b_x - a_x.b_z, a_x.b_y - a_y.b_x)
        """
        a = cross_product_inputA
        b = cross_product_inputB
        cp_tensor = (((a[1][0] * b[2][0]) - (a[2][0] * b[1][0])),
                     ((a[2][0] * b[0][0]) - (a[0][0] * b[2][0])),
                     ((a[0][0] * b[1][0]) - (a[1][0] * b[0][0])))
        return cp_tensor

    def format_matrix(self,matrix):
        matrix_in_string = str(matrix)
        matrix_in_string = matrix_in_string.replace('],',',\n')
        matrix_in_string = matrix_in_string.replace('\n ','\n')
        matrix_in_string = matrix_in_string.replace(',','')
        matrix_in_string = matrix_in_string.replace('[','')
        matrix_in_string = matrix_in_string.replace(']','')
        return matrix_in_string

inn = "Matrix 2*2"
size = ((2,2))
inputA = Matrix_Create.create_matrix(inn, size)
inputB = Matrix_Create.create_matrix(inn, size)
print("Matrix 2x2 Input A is: ", inputA)
print("Matrix 2x2 Input B is: ", inputB)
Matrix_Exec = Matrix(inputA, inputB)
#print(Matrix_Exec.matrix_addition())
MatrixAdd = (Matrix_Exec.matrix_addORsub('+'))
print("2X2 Matrix Addition Result:", Matrix_Exec.format_matrix(MatrixAdd))

MatrixSub = (Matrix_Exec.matrix_addORsub('-'))
print("2X2 Matrix Subtraction Result:", Matrix_Exec.format_matrix(MatrixSub))

MatrixMultiply = Matrix_Exec.matrix_multiplication()
print("2x2 Matrix Multiplication Result:", Matrix_Exec.format_matrix(MatrixMultiply))

Determinant_Result = Matrix_Exec.determinant()
print("2x2 Matrix Determinant Result:", Matrix_Exec.format_matrix(Determinant_Result))

Inverse_Result = Matrix_Exec.inverse()
print("2x2 Matrix Inverse Result:", Matrix_Exec.format_matrix(Inverse_Result))

Cross_Product_Size = ((3, 1))
CR_PR_inputA = Matrix_Create.create_matrix(inn, Cross_Product_Size)
CR_PR_inputB = Matrix_Create.create_matrix(inn, Cross_Product_Size)
Cross_Product_Result = (Matrix_Exec.cross_product(CR_PR_inputA, CR_PR_inputB))
print("3x1 Cross_Product_InputA:", CR_PR_inputA)
print("3x1 Cross_Product_InputB:", CR_PR_inputB)
print("Cross_Product_Result:", Matrix_Exec.format_matrix(Cross_Product_Result))

Matrix 2x2 Input A is:  [[1, 2], [5, 1]]
Matrix 2x2 Input B is:  [[1, 2], [3, 5]]
2X2 Matrix Addition Result: 2 4
8 6
2X2 Matrix Subtraction Result: 0 0
2 -4
2x2 Matrix Multiplication Result: 7 12
8 15
2x2 Matrix Determinant Result: -9
2x2 Matrix Inverse Result: ((-0.1111111111111111 -0.2222222222222222) (-0.5555555555555556 -0.1111111111111111))
3x1 Cross_Product_InputA: [[1], [2], [1]]
3x1 Cross_Product_InputB: [[2], [3], [5]]
Cross_Product_Result: (7 -3 -1)
